# Schematics: Transparency tricks

Although `latticegen` is primarily focussed on generating numerical data to test algorithms (e.g. [GPA](https://github.com/TAdeJong/pyGPA)) on, it is also possible to use it to generate more schematic renderings of lattices.
To visualize bonds and combine different lattices, it is possible to play transparency tricks with `matplotlib`s `imshow()` and the diverging colormaps.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import latticegen

## A hexagonal moiré lattice

By applying a hexagonal lattice of `order=1` itself as transparency, we obtain something nicely respresenting the bonds in the lattice.
<div class="alert alert-info">
**Note:** In `matplotlib`, `alpha` values are clipped between 0 and 1. Here, the lattices are scaled to $max = 1$ instead of using the `normalize` kwarg of `anylattice_gen()`, to obtain slightly moire transparent lattices overall.
</div>

In [ ]:
r_k = 0.05
lattice1 = latticegen.hexlattice_gen(r_k, 0, 1)
lattice1 = (lattice1 / lattice1.max()).compute()
lattice2 = latticegen.hexlattice_gen(r_k, 5, 1)
lattice2 = (lattice2 / lattice2.max()).compute()

fig,axs = plt.subplots(ncols=3, figsize=[10,4])
for i in [0, 1]:
    axs[i].imshow(-lattice1.T, cmap='PiYG', 
                      vmax=1,
                      vmin=-1,
                       alpha=lattice1.T
                     )
    axs[i + 1].imshow(lattice2.T, cmap='PiYG',
                   vmax=1,
                   vmin=-1,
                   alpha=lattice2.T)

## A diatomic lattice

By combining two separate trigonal lattices, we can illustrate a diatomic hexagonal lattice such as the insulator hexagonal Boron Nitride (hBN). A different colormap is used to generate different colors. 

(Any combination of colors can be obtained by [creating a custom colormap](https://matplotlib.org/stable/tutorials/colors/colormap-manipulation.html#directly-creating-a-segmented-colormap-from-a-list))

In [ ]:
r_hBN = r_k * (0.246 / 0.2504)
sublattice_a = latticegen.trilattice_gen(r_hBN, 0, order=1, normalize=True)
sublattice_a = sublattice_a.compute()
# Now add the second shifted sublattice lattice to get a hexagonal lattice
ks = latticegen.generate_ks(r_hBN, 0, sym=6)
x = np.array([ks[1], -ks[2]])
shift = (np.linalg.inv(x / r_hBN).T/(3*r_k)).sum(axis=0).T  # Don't ask, this works
sublattice_b = latticegen.trilattice_gen(r_hBN, 0, order=1,
                                         shift=shift, normalize=True)
sublattice_b = sublattice_b.compute()

fig, axs = plt.subplots(ncols=3, figsize=[10,4])
axs[0].set_title('Sublattice a')
axs[1].set_title('Both sublattices')
axs[2].set_title('Sublattice b')
for i in [0, 1]:
    axs[i].imshow(-sublattice_a.T, cmap='bwr', 
                  vmax=1, vmin=-1,
                  alpha=sublattice_a.T)
    axs[i + 1].imshow(sublattice_b.T, cmap='bwr',
                      vmax=1, vmin=-1,
                      alpha=sublattice_b.T)

## A moiré of a diatomic lattice and a hexagonal lattice

Putting both examples together to create an image of a moiré of a graphene lattice (green) on top of a hBN lattice (red/blue):

In [ ]:
plt.figure(figsize=[10,10])
plt.imshow(-sublattice_a.T, cmap='bwr', 
           vmax=1, vmin=-1,
           alpha=sublattice_a.T)
plt.imshow(sublattice_b.T, cmap='bwr',
           vmax=1, vmin=-1,
           alpha=sublattice_b.T)
plt.imshow(lattice2.T, cmap='PiYG',
           vmax=1, vmin=-1,
           alpha=lattice2.T*0.9)